# Nanodegree Engenheiro de Machine Learning

## Projeto Final Capstone

### Otimização de Campanha de Marketing via Clusterização e Regressão 

O objetivo do projeto é aplicar o aprendizado obtido durante o programa de Nanodegree Engenheiro de Machine Learning para solucionar um problema do mundo real em uma área de interesse ao aplicar algoritmos e técnicas de machine learning.

* Definicão do problema;
* Análise Exploratória de dados;
* Implementação do algoritmo;
* Avalição e Validação do modelo;
* Resultados;
* Conclusões;

### Definicão do problema
No Marketing digital, uma atividade muito comum é a utilização de ferramentas de anúncios que realizam campanhas tanto em sites de busca (Google, Bing, etc) ou em redes sociais (Facebook, Instagram,etc), em resumo, esses anúncios aparecem para as pessoas que estão buscando algum serviço em questão. As plataformas fornecem opções de filtros para que esses anúncios tenham um público-alvo espesifico, a pergunta feita é como as empresas sabem o perfil desses público alvo? Esse é o problema em questão que esse projeto irá ajudar a resolver.

#### Conjunto de Dados e Inputs

Os dados usados neste projeto foram encontrados no banco de dados da kaggle para infomações e download do dataset está [aqui.](https://www.kaggle.com/loveall/clicks-conversion-tracking)
Os dados se referem a uma da campanha publicitária de mídia social de uma organização anônima. O Dataset contém 1143 observações (Linhas) em 11 variáveis (Colunas). Abaixo estão as descrições das variáveis:

* **ad_id:** ID exclusivo para cada anúncio; 
* **xyz_campaign_id:** ID associado a cada campanha publicitária empresa XYZ;
* **fb_campaign_id:** ID associado a campanha como o facebook rastreia a camapanha;
* **age:** Idade da pessoa a quem o anúncio foi mostrado; 
* **gender:** Gênero da pessoa a quem o anúncio foi mostrado;
* **interest:** Código que especifica a categoria de qual o interesse da pessoa pertence; 
* **Impressions:** Numero de vezes que aunncio foi mostrado;
* **Clicks:** Número de cliques do anuncio;
* **Spent:** Quantidade pago pela empresa XYZ para o Facebook, para mostrar o anúncio;
* **Total conversion:** Número total de pessoas que se interessaram sobre o produto ou serviço depois de ver o anúncio;
* **Approved conversion:** Número total de pessoas que compraram o produto depois de ver o anúncio.

### Análise Exploratória de Dados

In [ ]:
# Importe as biblotecas
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
from kmodes.kprototypes import KPrototypes
from sklearn import preprocessing
from sklearn import cluster
from sklearn import decomposition
from sklearn import metrics
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression

In [ ]:
# Carrege os dados
data = pd.read_csv('KAG_conversion_data.csv')
# Criando uma copia dos dados (Boa Prática)
df = data.copy()

In [ ]:
# Visão geral dos dados
df.info()

**Obs 1:**  Aparentemente não tem nenhum dado faltando, os tipos de dados deveram ser investigados para validação

In [ ]:
# Inspecione as 5 primeiros observações
df.head()

Inspecionando os primeiros elementos, podemos perceber que a colunas 'age' e 'gender' são categóricas, apesar que a coluna 'age' ser numérica, ela representa uma categoria de uma faixa de idade, vamos investigar se alguma variável numérica é na verdade uma categoria

In [ ]:
# PLote grárfico de barra horizontal da quantidade de valores unicos de todas as variavéis númericas
plt.figure(figsize=[5,5])
for column in df._get_numeric_data():
    unique_prop = (len(df[column].unique())/df[column].size)*100
    plt.barh(y = column, width=unique_prop,color = 'b', alpha = 0.8)
plt.title("Contagem dos Valores únicos")
plt.xlabel("Soma")
plt.ylabel("Variáveis")

A visualização acima indica que 'Total_Conversion', 'Approved_Conversion', 'interest', 'xyz_campaign_id' podem ser categóricas, segundo o meta dados fornecidos,'Total_Conversion' e 'Approved_Conversion' seriam número total de conversões então seriam realmente dados numéricos, 'interest', 'interest' seriam códigos que especificam as categoria de interesse esses terão que ser alterados e 'xyz_campaign_id' seria o código da campanha da empresa XYZ também deverá ser alterado. 'ad_id'tem 100% de valores únicos o que significa que está servido como índice, vamos usar o índice do DafaFrame, estão essa coluna deverá ser removida, Em resumo, **será alterado o tipo das variáveis 'interest' e 'xyz_campaign_id' de numérico para categórico e 'ad_id' será removido.

In [ ]:
# Faça uma tabela frequência de 'xyz_campaign_id'
freq_table = pd.crosstab(df['xyz_campaign_id'],columns= 'Count')
freq_table['%'] = np.round(freq_table/freq_table.sum(),3)
freq_table.sort_values('%',ascending = False,inplace = True)

# Plote um gráfico de barras vertical
ax = freq_table['Count'].plot(kind= "bar",figsize=(10,7))

# Amazene o falor total das colunas a serem plotadas
total = freq_table["Count"].sum()

# Configure as legendas das colunas, eixos e titulo.
for i in ax.patches:
    # Ajuste a posição e tamanho da fonte da legenda
    ax.text(i.get_x()+.10, i.get_height()+.10, \
            str(round((i.get_height()/total)*100, 2))+'%', fontsize=15,
                color='dimgrey')
plt.title("Tabela Frequência de 'xyz_campaign_id'")    
plt.ylabel("Contagem")

Variável 'xyz_campaign_id' existem apenas 3 valores únicos, o que confirma que pode ser lidado como uma variável categórica, sendo divido entre as campanhas 1178 e 936, a campanha 916 representa apenas 4.72%.

In [ ]:
# Faça uma tabela frequência de 'interest'
freq_table = pd.crosstab(df['interest'],columns= 'Count')
freq_table['%'] = np.round(freq_table/freq_table.sum(),3)
freq_table.sort_values('%',ascending = False,inplace = True)

# Plote um gráfico de barras horizontal
ax = freq_table['Count'].plot(kind= "barh",figsize=(12,10))

# amazene o falor total das colunas a serem plotadas
total = freq_table["Count"].sum()

# Configure as legendas das colunas, eixos e titulo.
for i in ax.patches:
    # get_width pulls left or right; get_y pushes up or down
    ax.text(i.get_width()+.4, i.get_y()+.42, \
            str(round((i.get_width()/total)*100, 2))+'%', fontsize=13,
color='dimgrey')
ax.invert_yaxis()
plt.title("Tabela Frequência de 'interest")    
plt.ylabel("Código Interest")

Apesar de a distribuição sugere que é uma variável numérica, de acordo com os meta dados fornecidos `'interest'` é um código que especifica a categoria de qual o interesse da pessoa pertence, então vamos tratar-lo como uma variável categoria extensa, podemos essa variável como a matriz características dos clusteres.



In [ ]:
# Plote uma grafico e frequências das variáveis categoricas
plt.figure(figsize=[5,5])
cat_col = ['age','gender']
for column in cat_col:
    plt.figure()
    plt.title('Frequência "{}"'.format(column))
    plt.xlabel("Frequência")
    plt.ylabel("Categoria")
    freq_table = pd.crosstab(df[column],columns= ['Frequency(%)'], normalize = True)
    freq_table.sort_values('Frequency(%)',ascending = True,inplace = True)
    plt.barh(y = freq_table.index, width=freq_table['Frequency(%)'])

Podemos afirmar que pelo menos 80% dados são representados por pessoas de 30 a 39 anos, e é dividido entre homens e mulheres, com a maioridade com idade entre 30 a 34 anos.

In [ ]:
# Mostre as estatiscas descritivas
df.describe().iloc[:,5:]

Podemos observar que em 'Clicks', 'Spent', 'Total_Conversion', 'Approved_Conversion' apresentam uma grande diferença entre 75 percentil para o valor máximo, isso significa se ordenarmos nossos dados de forma crescente 75% deles seriam menor que o valor apresentado no linha "75%", no caso de "Clicks", seria 37.5 e ultimo valor máximo, o último valor dos dados ordenados, seria 421, é notável que houve um crescimento fora do padrão, considerando que ele saiu de 0 a 35.5 recorrendo 75%, claramente temos  outlies, teremos que investigar mais essa variáveis de forma visual.

In [ ]:
# Plote uma matriz de scatterplots e histogramas
sns.pairplot(df)

Agora ficou claro na presença de outlies nas variáveis `'Clicks'`, `'Spent'`, `'Total_Conversion'`, `'Approved_Conversion'` conforme mostra a curva assimétrica direita, confirmando a análise anterior, além disso todas elas aparetam ter alguma fortes correlações.

É possivel confirmar que a variável 'interest' é categorica por apresentar nenhuma correlacão com as demais.

In [ ]:
# Para visuzliar melhor as correlações, plote um mapa de calor das mesmas.
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')


Existem algumas variáveis que são fortemente correlacionadas, isso é notavel quando visualizamos os gráficos de dispersão que aparentam ser uma linha, que em outras palavras , indica o quanto a variável explica o comportamento linear da outra, para um problema de aprendizagem não supervisionada isso não é incessante, que implica que as variáveis tem a mesma informação, teremos que criar novas variáveis de forma que diminua essa coração e aumente a informação.

'fb_campaign_id' é muito correlacinada com 'ad_id', como anteriormente havimos definido a remocão de 'ad_id', será removido também 'fb_campaign_id'.

### Pré-processamento dos dados
Para organizar o Pré-processamento dos dados, essa atividade foi dividida 4 em partes.

**1 -** Alterar o tipo das variáveis 'interest' e 'xyz_campaign_id' de numérico para categórico e remoção 'ad_id' e 'fb_campaign_id'

**2 -** Criar novas variáveis utilizando 'Clicks' , 'Spent','Impressions'  e remover as variáveis numéricas inicias.

**3 -**  Escalonamento e Tratamento de Outlies.

**4 -** Seleção de atributos (Feature Selection).

### 1 -  Alterar o tipo das variáveis 'interest' e 'xyz_campaign_id' de numérico para categórico e remoção 'ad_id' e 'fb_campaign_id'.

#### Código:

In [ ]:
# Altere para 'category' todas as variáveis para manter um padrão
df[['xyz_campaign_id','age','interest','gender']] = df[['xyz_campaign_id','age','interest','gender']].astype('category')

# Remova'ad_id' e 'fb_campaign_id'
df.drop(['ad_id','fb_campaign_id'], axis = 1, inplace = True)

# Slave as variávies categoricas
cat_var = df.select_dtypes(include = 'category').columns.tolist()

# Amazene as variáveis numéricas
num_var = df._get_numeric_data().columns

#### Teste:

In [ ]:
# Inspecione o resultado
display(df.info())
print("Lista das variáveis categóricas:")
display(cat_var)
print("Lista das variáveis numéricas:")
display(num_var)

### 2- Criar nova variável utlizando 'Clicks' , 'Spent','Impressions'  e tratar outlies se necessário.

Seram criados alguns KPI padrões que são utilizados no marketing digital:

**Taxa de cliques "Click-Through-Rate" (CTR):** Esta é a porcentagem de quantas impressões se tornaram cliques. Mede a atratividade do anúncio. Um Benchmark para essa KPI seria 2% para ser razoável.

**Taxa de Conversão "Conversion-Rate" (CR):** Esta é a porcentagem de cliques que resultam em uma "conversão". Conversão é determinado por um objetivo que é definido para a campanha. O que mede efetivamente a efetividade anúncio de campanha. Para dataset do projeto, serão criados 2 taxas CR por temos 2 variáveis distintas de conversão, uma que o objetivo é despertar o interesse e a outra em comprar o produto/serviço.



#### Code:

In [ ]:
# Taxa de cliques "Click-Through-Rate"
df['CTR'] = (df['Clicks'] / df['Impressions'])*100

# Taxa de Conversão "Conversion-Rate" para 'Approved_Conversion'
df['ACR'] = ((df['Approved_Conversion'])/ df['Clicks'])*100

# Taxa de Conversão "Conversion-Rate" para 'Total_Conversion'
df['TCR'] = ((df['Total_Conversion'])/ df['Clicks'])*100


df.replace([np.inf, -np.inf], [100,0], inplace = True)

# Atualize Num_var
num_var= ["CTR","ACR","TCR"]

df[num_var] = df[num_var].fillna(0)

#### Teste:

In [ ]:
df.head()

In [ ]:
corr = df._get_numeric_data().corr()
corr.style.background_gradient(cmap='coolwarm')

O Problema das correlações não foi resolvido e temos outlies, mas tivemos um bom resultado com as distribuições e correlações das variáveis criadas. Como temos 3 Variáveis altamente correlacionadas entre elas, e algorimitmos de clusterização cometer erros por viés, devemos escolher uma, como estamos objetivando melhorar a taxa de converção, a variável mais adequeada entre as 3, seria a "Impressions", de forma de nos entendermos como o comusmidor se comportar ao clicar nos anuncios.

In [ ]:
df.describe()

### 3.1 -  Tratando Outlies

Essa atividade será divida em 3 partes:

- Identificação de outliers: Será utilizado Bloxplot para visualizar esses pontos, lembrando que Boxplot por padrão, identifica como outlier as pontos que estão a 1.5 desvios padrões do IQR(Intervalo inter Quartil).
- Análise: Será avaliado se esses pontos fazem sentido, ver a representatividade  dos dados
- Tratamento: Será decidido os pontos serram mantidos, alterados ou removidos nessa ordem de importância, o escalonamento deve acontecer para reduzir também reduzir o efeito de outlies


#### 3.1.1 Indentificacão

In [ ]:
# Plote um boxplot das variáveis numéricas
sns.boxplot(data = df[num_var])
plt.title("Indentificando Outlies")

Aparemente apenas "CTR" tem alguns outlies, devemos pegar os seus indices e explorálos

In [ ]:
# Encontre os outlies, mostre e amazerne os índices
out_idx = []


for feature in num_var:
       
    # Calule a faxa onde estão os outlies (+- 1.5*IQR)
    Q1 =  df[feature].quantile(.25)
    Q3 = df[feature].quantile(.75)
    IQR = Q3- Q1
    out_min = Q1 - 1.5*IQR
    out_max = Q3 + 1.5*IQR

    # Amarzene os indices dos outlies   
    idx = df[feature][(df[feature] <= out_min) | (df[feature] >= out_max)].index.tolist()
    out_idx += idx

    # Calcule a proporção dos outlies
    out_p = np.round((len(idx) / len(df)) * 100,2)

    # Mostre para inspeção os outlies encontrados
    print("{} ({}%) observações consideradas outlies da variável {}".format(len(idx),out_p,feature))
    display(df.loc[idx].head(3))
    display(df.loc[idx,feature].describe())

# Transfore a lista em um Panda Series
out_idx = pd.Series(out_idx)
    
# Encontre os valores duplicados
dup_idx = out_idx.iloc[np.where(pd.Series(out_idx).duplicated(keep=False))[0]]
dup_idx = dup_idx.unique()

# Encontre os valores únicos
out_idx = out_idx.unique()

# Calcule a representação total dos outlies no data set
out_p = np.round((len(out_idx) / len(df)) * 100,2)
dup_p = np.round((len(dup_idx) / len(df)) * 100,2)

# Mostre a quantidade total de outlies
print("No total, {} ({}%) linhas são outlies unicos, sendo {} ({}%) em pelo menos 1 variável".format(len(out_idx),out_p,
                                                                                                   len(dup_idx),dup_p)) 

Existe  uma quantidade expressiva de outliers, remover-los irá impacatar demais no dataset, além disso, eles aparatam ser dados legitimos, talvez algumas camapanhas que estão rodando a mais tempo e por isso receberam mais impressions e clicks. Esses outlies seram tratados de uma forma matmática, de forma que seram colocados em uma escalados logarítima, onde irá diminuir a diferença dos valores extremos, mas primeiro temos que tratar os zeros do data set, na escala logaritima, a valor zero vale menos infito ( - inf)

In [ ]:
df[num_var].describe()

In [ ]:
# Substitua os zeros do dataset por um número muito pequeno (0,01)
for column in df[num_var].columns:
    df.loc[df[column] <= 0.01,column] = 0.1

# Transforme as novas numéricas (velhas e novas) na escala logarítima
df[num_var] = np.log(df[num_var] )

In [ ]:
df[num_var].describe()

In [ ]:
sns.boxplot(data = df[num_var])

In [ ]:
# Encontre os outlies, mostre e amazerne os índices
out_idx = []

for feature in num_var:
       
    # Calule a faxa onde estão os outlies (+- 1.5*IQR)
    Q1 =  df[feature].quantile(.25)
    Q3 = df[feature].quantile(.75)
    IQR = Q3- Q1
    out_min = Q1 - 1.5*IQR
    out_max = Q3 + 1.5*IQR

    # Amarzene os indices dos outlies   
    idx = df[feature][(df[feature] <= out_min) | (df[feature] >= out_max)].index.tolist()
    out_idx += idx

    # Calcule a proporção dos outlies
    out_p = np.round((len(idx) / len(df)) * 100,2)

    # Mostre para inspeção os outlies encontrados
    print("{} ({}%) observações consideradas outlies da variável {}".format(len(idx),out_p,feature))
    display(df.loc[idx].head(3))
    display(df.loc[idx,feature].describe())

# Transfore a lista em um Panda Series
out_idx = pd.Series(out_idx)
    
# Encontre os valores duplicados
dup_idx = out_idx.iloc[np.where(pd.Series(out_idx).duplicated(keep=False))[0]]
dup_idx = dup_idx.unique()

# Encontre os valores únicos
out_idx = out_idx.unique()

# Calcule a representação total dos outlies no data set
out_p = np.round((len(out_idx) / len(df)) * 100,2)
dup_p = np.round((len(dup_idx) / len(df)) * 100,2)

# Mostre a quantidade total de outlies
print("No total, {} ({}%) linhas são outlies unicos, sendo {} ({}%) em pelo menos 1 variável".format(len(out_idx),out_p,
                                                                                                   len(dup_idx),dup_p)) 

Podemos percerber, a técnica aplicada foi extremamente eficaz, nossa proporção de outlies cairam de 41.99%% para 9.62%, apesar de quase 10% é o valor expresivo, o fato de após ter colocato em escala logarítima as os valores ainda se apresetam como outlies, eles deveram ser removidos

In [ ]:
df.drop(out_idx,inplace =True)

In [ ]:
corr = df._get_numeric_data().corr()
corr.style.background_gradient(cmap='coolwarm')

Após o tratamento dos outlies, as correlações melhoraram bastante, podemos procesguir com o pre processamento dos dados para treinar o algoritimo.

### Feature Selection

Para melhorar a performance do modelo e evitar que ele possa sofrer erro por viés de uma variável que esteja altamente correlacionada, iremos testar as variáveis numéricas usando a seguinte metodologia:

Vamos remover uma variável numérica e treinar um modelo de regressão para tentar prever utilizando as variáveis restantes, iremos validar o modelo utilizando a métrica R^2 ( coeficiente de determinação) que mede o quão bom está o modelo, se o modelo tiver um alto R2 significa que a variável está perfeitamente sendo prevista , o que implica que ela não é relevante para identificar caraterísticas únicas, já que outras variáveis tenham a mesma informação que a mesma.

In [ ]:
# Use sklearn to perform a train-test split on data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from sklearn.metrics import r2_score

for columun in num_var:
    X = df[num_var].drop(columun, axis =1).values
    y = df[columun].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =15)


    # Train the model
    model = LinearRegression()

    # Fit the model
    model.fit(X_train,y_train)
    
    # Get Predictions
    predictions = model.predict(X_test)
          
    # Print the results
    print('R^2: ' + str(columun) +" "+ str(r2_score(y_test, predictions)))


Não há nenhuma variável que foi completamente prevista pelas outras, o que significa que o modelo corre menos risco de sofrer erro de viés por alguma variável.

O mesmo modelo de teste será realizado com as variáveis categorias, mas agora utilizando o teste de independência Qui-Quadrado é usado para descobrir se existe uma relação entre as variáveis categóricas, enquanto menor o valor (próximo de zero) siginfica que uma váreial está extremamente relacionada a outra.

In [ ]:
from scipy.stats import chi2_contingency

chi_matrix = []
for category_1 in cat_var:
    chi_list = []
    for category_2 in cat_var:
        observed = pd.crosstab(df[category_1],df[category_2])
        chi_list.append(chi2_contingency(observed)[1])
        
    chi_matrix.append(chi_list)
chi_matrix = pd.DataFrame(chi_matrix,index= cat_var,columns=cat_var)
chi_matrix

Avaliando os resultados do teste Qui-Quadrado, pode observar que as variáveis "xyz_campaign_id" e "gender" tem relação entre todas as variáveis, sendo assim, selas seram removidas para que o modelo não sofra viés.

In [ ]:
# Atualize cat_var
cat_var = df.drop(["xyz_campaign_id","gender"],axis =1).select_dtypes(include = 'category').columns.tolist()

### Implementação

####  Kprototype

In [ ]:
cat_var

In [ ]:
# Selecione as colunas que seram treinadas
train_clolmuns = num_var + cat_var

# Transforme os dados em arrays para melhor processamento
X = df[train_clolmuns].values

# Crie uma lista para amarzenar os valores do coeficiente shioueta
avg_silhouettes = [] 

# Treine modelo com 2 a 6 clusters
for k in range(2,6):
        kp = KPrototypes(n_clusters = k, random_state=28) # Crie a instância
        cluster_labels = kp.fit_predict(X,categorical=[3,4]) # Pege os labels dos clusters
        shihouette_avg = metrics.silhouette_score(X[:,:-2], cluster_labels) # Calcule o coeficiente de silhueta
        avg_silhouettes.append(shihouette_avg) # Amarzene o coeficiente de silhueta
        
# Plote o gráfico de silhueta        
plt.figure() 
plt.title("Coeficiente de Silhueta K-Protopypes")
plt.xlabel("Número de K")
plt.ylabel("Valor Silhueta")
plt.xticks(np.arange(2, 6, step=1))
plt.plot(list(range(2,6)),avg_silhouettes)

# Crie e treine o modelo otimizado
kp = KPrototypes(n_clusters= np.argmax(avg_silhouettes) + 2,random_state=28)
df['cluster'] = kp.fit_predict(X,categorical=[3,4])

# Reduza para 2 dimensões
pca_column = num_var
pca = decomposition.PCA(n_components=2,random_state=28)
df['pc1'], df['pc2'] = zip(*pca.fit_transform(df[pca_column]))


# Crie listas para configurar os marcadores e cores das visualização
plt.figure()
markers = ["o","s","^","8","x"]
colors =  ["b","g","r","c","m"]

# Crie um loop para plotar os cluters na dimesão reduzida
plt.figure()
for c in df['cluster'].unique():
    table = df[df['cluster'] == c]
    plt.scatter(table['pc1'],table['pc2'], marker=markers[c], color=colors[c])
    plt.title('K = {}'.format(np.argmax(avg_silhouettes) + 2))
    plt.xlabel('PC1')
    plt.ylabel('PC2')
print("{}% da variância é explicada por PC1 e {}% por PC2 totalizando {}% da variância dos dados".format(round(pca.explained_variance_ratio_[0]*100,2),round(pca.explained_variance_ratio_[1]*100,2),
                                                                                                       round(sum(pca.explained_variance_ratio_)*100,2)))

In [ ]:
print("O modelo final de clusterização obeteve o Coeficiente de Silhueta: " + str(np.max(avg_silhouettes)))

Observando os resultado, é possível identificar facilmente os 3 grupos , essa visualização está bastante confiável por representar 86% da variância dos dados.

Para Estudar qual cluster é mais interessante para o negócio, uma regressão linear predizendo o *'ACR'* e analisar os seus coeficientes.

Primeiro devemos voltar os dados que foram colocados na escala logarítima para eu estado normal utilizando da função exponencial

In [ ]:
# Retonar a escala original 
df[num_var] = np.exp(df[num_var])
df[num_var].head()

Utilizamos o PCA para visualisar os nosso dados, agora podemos remover essas colunas

In [ ]:
# Utilizando
df.drop(["pc1","pc2"], axis =1 , inplace = True)

In [ ]:
df.drop("cluster", axis = 1).select_dtypes(exclude = ["uint8"])


Para entender qual o melhor cluster, vamos criar uma nova variável **ROAS** (Retorno Sobre o Investimento Publicitário), que seria a métrica que avalia a viabilidade econômica da campanha, ela é calculada pela razão da receita pelo investimento

In [ ]:
# Calule o ROAS
df["ROAS_rate"] = (df["Approved_Conversion"] /df["Spent"])*100

# Ajuste os dados que sofreram uma diviSão por 0
df.replace([np.inf, -np.inf], [1,0], inplace = True)
df["ROAS_rate"] = df["ROAS_rate"].fillna(0)
df['ROAS_rate'] = df['ROAS_rate'].astype('int64')

Para o melhor entendimento dos dados, vamos explorar raptamentes as variáveis de cada cluster

In [ ]:
# Criado array com as médias de cada coluna
total_mean = df.drop(["cluster"], axis = 1).select_dtypes(exclude = ["uint8","category"]).mean()
total_mean = np.log(total_mean)
# Criando visualização com a diferença de cada coluna
for i in range(3):
    plt.figure(i,figsize = [12,4])
    plt.ylim(-3,2)
   
    plt.title("Cluster {}".format(i))
    df_cluster = df.drop("cluster", axis = 1).select_dtypes(exclude = ["uint8"])[df["cluster"]==i]
    cluster_mean = df_cluster.mean()
    cluster_mean = np.log(cluster_mean)
    
    cluster_diff = cluster_mean - total_mean
    plt.bar(cluster_diff.index, cluster_diff)
    plt.xlabel('Variavél')
    plt.ylabel('Diferença da média')

É possível perceber que o cluster 2 tem uma grande disparidade com a média geral em relação ao *'ACR'* e fica muito próximo da média nas outras variáveis, fica claro que esse grupo não tem nenhuma `'Approved_Conversion'`. No cluster 0 e 1, temos `'Approved_Conversion'` sendo o 1 com valores muito acima da média porém com custo maior e `ACR`, podendo ser interpretado casos pouco eficiência já que se precisou de muito mais 'clicks' e consequentemente mais `'Spent'`, o cluster 0 temos bem menos  de média em `'Approved_Conversion'` porém ao considerável menor custo, indicando uma boa eficiência O ROAS do cluster 0 e 1 estão opostos (acima e abaixo da média), inicialmente podemos acreditar que o cluster 0 é o mais indicado para receber investimento já que mostra que o ROAS acima da média.

Vamos tratar os clusters como categorias e por conta disso vamos utilizaar a técnica de one hot encondig para transformar em diferentes variáveis binárias, dessa forma é possivel incluir no modelo de regressão

#### Regressão Linear e Árvore de Decisão Aleatória.

In [ ]:
# Crie variável dumie para os clusters e remova 1
df = pd.concat([df,pd.get_dummies(df['cluster']).drop(2, axis =1)],axis = 1)
# Remova A coluna "Cluster"
df.drop("cluster", axis = 1, inplace = True)
df['ROAS_rate'] = df['ROAS_rate'].astype('int64')

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
# Selecione as variáveis preditores e alvo
training_columns = ["Spent","Approved_Conversion","ACR","CTR","TCR",0,1]
label_column = "ROAS_rate"

In [ ]:
# Transform em array as variáveis alvo e preditora
X = df[training_columns].values
y = df[label_column].values

# Divida os dados em cojunto de treino teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 28)
clf = LassoCV(cv = 10,normalize = True ,random_state = 28)
clf.fit(X_train, y_train)

# Realize as predicões
pred = clf.predict(X_test)

# Imprima os coeficientes e as métricas de avaliação
print(clf.coef_)
print('RMSE: ' + str(mean_squared_error(pred ,y_test)**0.5))
print(pearsonr(pred,y_test))
print(clf.get_params)
    

In [ ]:
# Cria uma dicionários com os hyperparametros a serem testados do Random Forest
param_grid = { 
    'n_estimators': [100,200],
    'max_depth' : [4,5,6,7],
    'criterion' :['gini', 'entropy']
}

# Instancie o modelo em uma váriavel
rfc = RandomForestClassifier(random_state = 28)
clf = GridSearchCV(estimator = rfc, param_grid = param_grid, cv = 10,scoring  = 'r2')
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

# Imprima os resultados
print('RMSE: ' + str(mean_squared_error(pred ,y_test)**0.5))
print(pearsonr(pred,y_test))
print(clf.get_params)
print(clf.best_params_)    